In [1]:
import fsspec
import pandas as pd

In [2]:
host='carnation.dbs.umt.edu'
port=2229
user=''
pw=''


fs = fsspec.filesystem('ssh', host=host, port=port, username=user, password=pw)

In [3]:
with fs.open('./YNP_RNA/raw_data/fastqs/Mguttatus_256_v2.0.gene_exons.gff3') as f:
    df = pd.read_csv(f, sep='\t', names=['chromosome','source','feature_type','start_pos','end_pos', 'score','strand','frame','attribute'],skiprows=2)

with fs.open('./YNP_RNA/raw_data/fastqs/Mguttatus_256_v2.0.gene_exons.gff3') as f:
    df1 = pd.read_csv(f, sep='\t', names=['chromosome','source','feature_type','start_pos','end_pos', 'score','strand','frame','attribute'],skiprows=2)

# string split the attribute tab 

def append_ah_suffix(df: pd.DataFrame, suffix: str):
    ndf = df[df['feature_type'].isin(['mRNA', 'gene'])].attribute.str.split(r'(?<=Name)(.*?)(?=;)',expand=True, regex=True)
    df['attribute'][df['feature_type'].isin(['mRNA', 'gene'])] =  ndf.iloc[:,0] + ndf.iloc[:,1] + suffix + ndf.iloc[:,2]
    return df 

df['chromosome'] = df['chromosome'] + '_AHQT'
df1['chromosome'] = df1['chromosome'] + '_AHQN'

df = append_ah_suffix(df, suffix='_AHQT')
d1 = append_ah_suffix(df1, suffix = '_AHQN')


frames = [df, df1]
df_merge = pd.concat(frames)

/var/folders/d0/88jy7tpj3f56hk881nnmn_6r0000gn/T/ipykernel_88747/2598409965.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['attribute'][df['feature_type'].isin(['mRNA', 'gene'])] =  ndf.iloc[:,0] + ndf.iloc[:,1] + suffix + ndf.iloc[:,2]


In [4]:
with fs.open('./YNP_RNA/pseudo.gff3', 'w') as fp: 
   fp.write('##gff-version 3\n##annot-version v2.0\n')
   df_merge.to_csv(fp, sep='\t', index=False, header=False)



